In [1]:
Company.objects.all()

list(Company.objects.all())

[<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>,
 <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>,
 <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>,
 <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>,
 <Company: 90041917-e41e-46f2-b979-7d3ad53efad2: Engie Umburanas>,
 <Company: 69c4238a-fc42-4a16-a312-364ff4cd10c6: GSB - UHE-ITA>,
 <Company: 4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHADINHO>,
 <Company: 1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: GSO - PCH-RONDONÓPOLIS>,
 <Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>,
 <Company: 6ea3a49c-55a7-4aeb-919f-949034a5d2fa: GSB - UHE-ESTREITO>,
 <Company: e2d725b7-1b3b-4138-b790-cf49a6ab5284: GSB - UHE-JAGUARA>,
 <Company: 6af632a4-8b09-4bc2-bc6c-43d9a7dc23f4: GSB - UHE-MIRANDA>,
 <Company: 76ddb208-b049-4022-9736-9fd0101a5392: GSB - UHE-PONTE DE PEDRA>,
 <Company: 247c952a-9371-483d-8039-9a65b41b8db3: GSB - UHE-SÃO SALVADOR>,
 <Company: 24072e01-a4a7

In [2]:
!cat ../../.env
# !cat '''path_to_the_file(.env)'''

cat: ../../.env: No such file or directory


In [3]:
import numpy
from numpy import arange
from helpers.km_converter import km_to_coordinates
from collections import defaultdict
from openpyxl import Workbook
from django.db.models import Q
from tqdm.notebook import tqdm
import datetime as dt
import pytz
from apps.service_orders.const import resource_approval_status
from django.db.models import CharField
from django.db.models.functions import Cast
import pandas as pd

In [4]:
list(Company.objects.filter(name__startswith='GES').values_list('name',flat=True))

['GES - UHE-CANA BRAVA',
 'GES - UHE-ESTREITO',
 'GES - PCH-RONDONÓPOLIS',
 'GES - UHE-MACHADINHO',
 'GES - UHE-SALTO SANTIAGO',
 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
 'GES - UHE-SÃO SALVADOR',
 'GES - GEH',
 'GES - UHE-ITA',
 'GES - UHE-PASSO FUNDO',
 'GES - UHE-SALTO OSORIO',
 'GES - UHE-MIRANDA',
 'GES - UHE-PONTE DE PEDRA',
 'GES - UHE-JAGUARA']

In [5]:
list(Company.objects.filter(name__startswith='GES').annotate(uuid_str=Cast("uuid",output_field=CharField())).values_list('uuid_str',flat=True))

['cff59760-69dd-4d07-bba5-61aa8b6faeea',
 '42ddc389-3258-4daf-868a-097235427e0f',
 '97f7268b-32f8-434f-8254-ecd647a60e37',
 'b4a06d54-e99b-4a79-82e5-2460a06fcbc5',
 '94fc0888-2e78-43c9-ac2d-f6dffc86d5fc',
 '447a03e3-27ea-4bb1-992b-d3d8784974cd',
 '96bcaf1a-593b-4c6e-bdca-9b0dad0db0f1',
 '68e3af24-2fec-4ab0-b2ff-97e1076b02b3',
 'd436bd3c-e145-44f2-a5ee-797a67110efd',
 '574400b8-28b9-4a59-b29f-9d542e42fbdc',
 '3167c3b2-95fa-4ecb-b759-dc1d63067ba4',
 '2fd0ca76-b741-47d7-af9c-607e6ed1eca2',
 '296edae1-b154-40ff-87b1-3b78e3441906',
 '17663a30-5f71-4e7c-8ac2-adf472342b42']

In [29]:
usinas = ['GES - PCH-JOSÉ GELÁZIO DA ROCHA', 
'GES - PCH-RONDONÓPOLIS', 
'GES - UHE-CANA BRAVA', 
'GES - UHE-ESTREITO', 
'GES - UHE-ITA', 
'GES - UHE-JAGUARA',
'GES - UHE-MACHADINHO',
'GES - UHE-MIRANDA',
'GES - UHE-PASSO FUNDO',
'GES - UHE-PONTE DE PEDRA',
'GES - UHE-SALTO OSORIO',
'GES - UHE-SALTO SANTIAGO',
'GES - UHE-SÃO SALVADOR',
'GES - GEH']

In [6]:
# today = dt.date.today().strftime('%m')
today = 5
ano = dt.date.today().year

In [7]:
months_number = [i for i in range(1, int(today)+1)]

In [8]:
months_number

[1, 2, 3, 4, 5]

In [30]:
def get_permission(obj, company):
    try:            
        user_in_company = next(
            a
            for a in obj.companies_membership.all()
            if a.company == company
        )
        return user_in_company.permissions.name
    except Exception:
        return None

In [31]:
def get_total_price(obj):
    total_price = 0
    for resource in obj.resources.all():
        try:
            resource_total_price = resource.unit_price * resource.amount
            if not isinstance(resource_total_price, (int, float)):
                raise Exception()
            total_price += resource_total_price
        except Exception:
            continue

    return total_price

In [32]:
def get_spent_price(obj):

    spent_price = 0
    for resource in obj.resources.all():
        for procedure_resource in resource.serviceorderresource_procedures.all():
            try:
                resource_spent_price = ((procedure_resource.unit_price * procedure_resource.amount) if procedure_resource.approval_status == resource_approval_status.APPROVED_APPROVAL else 0)
                if not isinstance(resource_spent_price, (int, float)):
                    raise Exception()
                spent_price += resource_spent_price
            except Exception:
                continue

    return spent_price

In [33]:
#vinculação de registros
def vinculacao_registro(months_name, company):
    vinc_regis = []
    for n in months_name:
        occurrences_total=OccurrenceRecord.objects.filter(company__name=company, created_at__year=ano, created_at__month=n, operational_control__isnull=True).count()
        occurrences_services=OccurrenceRecord.objects.filter(company__name=company, created_at__year=ano, created_at__month=n, operational_control__isnull=True, service_orders__isnull=False).count()
        try:
            vinc_regis.append(round(occurrences_services*100/occurrences_total, 2))
        except ZeroDivisionError:
            vinc_regis.append(0)
    return vinc_regis

In [34]:
#emissão de boletim de medição
def boletim_medicao(months_name, company):
    boletim = []
    for n in months_name:
        bulletin_count=MeasurementBulletin.objects.filter(Q(contract__firm__company__name=company)|Q(contract__subcompany__company__name=company), creation_date__year=ano, creation_date__month=n).count()
        boletim.append(bulletin_count)
    return boletim

In [35]:
# Número de rg homologados
def rg_homologados(months_name, company):
    num_homologados = []
    for n in months_name:
        homologado_count=len(list(set(OccurrenceRecord.history.model.objects.filter(company__name=company, history_date__year=ano, history_date__month=n, approval_step__name__icontains='deferido').values_list('uuid'))))
        num_homologados.append(homologado_count)
    return num_homologados

In [36]:
# Número de serviços
def num_servicos(months_name, company):
    servicos = []
    for n in months_name:
        services=ServiceOrder.objects.filter(company__name=company, opened_at__year=ano, opened_at__month=n).count()
        servicos.append(services)
    return servicos

In [37]:
# Número de registros operacionais
def rg_operacionais(months_name, company):
    num_operacionais = []
    for n in months_name:
        occurrences_operational=OccurrenceRecord.objects.filter(company__name=company, created_at__year=ano, created_at__month=n, operational_control__isnull=False).count()
        num_operacionais.append(occurrences_operational)
    return num_operacionais

In [38]:
# Número de rg ambiental
def rg_ambiental(months_name, company):
    num_ambiental = []
    for n in months_name:
        occurrences_ambiental=OccurrenceRecord.objects.filter(company__name=company, created_at__year=ano, created_at__month=n, operational_control__isnull=True, search_tags__name='Ambiental').count()
        num_ambiental.append(occurrences_ambiental)
    return num_ambiental

In [39]:
# Número de rg patrimonial
def rg_patrimonial(months_name, company):
    num_patrimonial = []
    for n in months_name:
        occurrences_ambiental=OccurrenceRecord.objects.filter(company__name=company, created_at__year=ano, created_at__month=n, operational_control__isnull=True, search_tags__name='Patrimonial').count()
        num_patrimonial.append(occurrences_ambiental)
    return num_patrimonial

Metas

In [40]:
vinculação_servicos = 10
emissao_boletim = 2
num_homologados= 12
numero_servicos = 60
num_operacionais = 40
num_ambiental = 5
num_patrimonial= 11

In [41]:
def nota_vinc (nota):
    print(nota)
    if nota >= 0.7*(vinculação_servicos):
        return 2 
    else:
        return -2

In [42]:
def nota_boletim (nota):
    if nota >= 0.7*(emissao_boletim):
        return 3 
    else:
        return -3

In [43]:
def nota_homologados (nota):
    if nota >= 0.7*(num_homologados):
        return 3 
    else:
        return -3

In [44]:
def nota_servicos (nota):
    if nota >= 0.7*(numero_servicos):
        return 1 
    else:
        return -1

In [45]:
def nota_operacional (nota):
    if nota >= 0.7*(num_operacionais):
        return 1 
    else:
        return -1

In [46]:
def nota_ambiental (nota):
    if nota >= 0.7*(num_ambiental):
        return 2 
    else:
        return -2

In [47]:
def nota_patrimonial (nota):
    if nota >= 0.7*(num_patrimonial):
        return 2 
    else:
        return -2

In [48]:
vinculacao = []
boletim = []
homologados = []
servicos = []
operacional = []
patrimonial = []
ambiental = []
for company in tqdm(usinas):
    print(company)
    vinculacao.append(vinculacao_registro(months_number, company))
    boletim.append(boletim_medicao(months_number, company))
    homologados.append(rg_homologados(months_number, company))
    servicos.append(num_servicos(months_number, company))
    operacional.append(rg_operacionais(months_number, company))
    patrimonial.append(rg_patrimonial(months_number, company))
    ambiental.append(rg_ambiental(months_number, company)) 
    print(f'Vinculação de registros: {vinculacao_registro(months_number, company)}')
    print(f'Emissão de boletim de medição: {boletim_medicao(months_number, company)}')
    print(f'Número de rg homologados: {rg_homologados(months_number, company)}')
    print(f'Número de Serviços: {num_servicos(months_number, company)}')
    print(f'Número de rg operacionais: {rg_operacionais(months_number, company)}')
    print(f'Número de rg patrimoniais: {rg_patrimonial(months_number, company)}')
    print(f'Número de rg ambientais: {rg_ambiental(months_number, company)}')

  0%|          | 0/14 [00:00<?, ?it/s]

GES - PCH-JOSÉ GELÁZIO DA ROCHA
Vinculação de registros: [0, 0, 0, 0, 0]
Emissão de boletim de medição: [1, 1, 1, 0, 0]
Número de rg homologados: [0, 0, 0, 0, 0]
Número de Serviços: [0, 0, 0, 0, 0]
Número de rg operacionais: [0, 0, 0, 0, 0]
Número de rg patrimoniais: [0, 0, 0, 0, 0]
Número de rg ambientais: [0, 0, 0, 0, 0]
GES - PCH-RONDONÓPOLIS
Vinculação de registros: [0, 0, 0, 0, 0]
Emissão de boletim de medição: [1, 1, 1, 0, 0]
Número de rg homologados: [0, 0, 0, 0, 0]
Número de Serviços: [0, 0, 0, 0, 0]
Número de rg operacionais: [0, 0, 0, 0, 0]
Número de rg patrimoniais: [0, 0, 0, 0, 0]
Número de rg ambientais: [0, 0, 0, 0, 0]
GES - UHE-CANA BRAVA
Vinculação de registros: [0.0, 13.33, 16.67, 0.0, 0]
Emissão de boletim de medição: [2, 0, 1, 1, 0]
Número de rg homologados: [1, 232, 200, 3, 0]
Número de Serviços: [0, 1, 2, 0, 0]
Número de rg operacionais: [68, 6, 13, 24, 0]
Número de rg patrimoniais: [23, 15, 12, 4, 0]
Número de rg ambientais: [0, 0, 0, 1, 0]
GES - UHE-ESTREITO
Vinc

In [49]:
hs_month = []
hs_month_export = {}
for i in tqdm(range(len(months_number))):
    hs = []
    for n in range(len(usinas)):
        soma = nota_vinc(vinculacao[n][i]) + nota_boletim(boletim[n][i]) + nota_homologados(homologados[n][i])+ nota_servicos(servicos[n][i]) + nota_ambiental(ambiental[n][i])+ nota_patrimonial(patrimonial[n][i]) + nota_operacional(operacional[n][i])
        hs.append((14+soma)*(100/28))
        if not hs_month_export.get(usinas[n]):
            hs_month_export[usinas[n]] = []
        hs_month_export[usinas[n]].append(hs[-1])

    hs_month.append(hs)

  0%|          | 0/5 [00:00<?, ?it/s]

0
0
0.0
0
58.33
14.29
4.69
91.67
0.0
0.0
0.0
0.0
0.0
0
0
0
13.33
0
38.89
25.0
0.0
100.0
0.0
0.0
96.5
0.0
0.0
0
0
0
16.67
0
66.67
12.5
0.0
100.0
97.11
0
0.0
0.0
0.0
0
0
0
0.0
0
70.0
0.0
0.0
0
42.86
0
28.57
0.0
0.0
0
0
0
0
0
0
0
0.0
0
0.0
0
0.0
0.0
0
0


In [50]:
hs_month_export

{'GES - PCH-JOSÉ GELÁZIO DA ROCHA': [0.0, 0.0, 0.0, 0.0, 0.0],
 'GES - PCH-RONDONÓPOLIS': [0.0, 0.0, 0.0, 0.0, 0.0],
 'GES - UHE-CANA BRAVA': [42.85714285714286, 50.0, 50.0, 0.0, 0.0],
 'GES - UHE-ESTREITO': [7.142857142857143, 0.0, 0.0, 0.0, 0.0],
 'GES - UHE-ITA': [57.142857142857146,
  78.57142857142857,
  78.57142857142857,
  57.142857142857146,
  0.0],
 'GES - UHE-JAGUARA': [14.285714285714286,
  78.57142857142857,
  57.142857142857146,
  21.42857142857143,
  0.0],
 'GES - UHE-MACHADINHO': [28.571428571428573,
  71.42857142857143,
  71.42857142857143,
  50.0,
  0.0],
 'GES - UHE-MIRANDA': [50.0,
  57.142857142857146,
  78.57142857142857,
  42.85714285714286,
  0.0],
 'GES - UHE-PASSO FUNDO': [21.42857142857143,
  21.42857142857143,
  57.142857142857146,
  50.0,
  0.0],
 'GES - UHE-PONTE DE PEDRA': [0.0, 14.285714285714286, 0.0, 0.0, 0.0],
 'GES - UHE-SALTO OSORIO': [0.0,
  71.42857142857143,
  14.285714285714286,
  64.28571428571429,
  21.42857142857143],
 'GES - UHE-SALTO SANTIAG

In [74]:
df2= pd.DataFrame.from_dict(hs_month_export,orient='index',columns=months_number)

In [78]:
df2=df2.round(2)
df2

,1,2,3,4,5
GES - PCH-JOSÉ GELÁZIO DA ROCHA,0.00,0.00,0.00,0.00,0.00
GES - PCH-RONDONÓPOLIS,0.00,0.00,0.00,0.00,0.00
GES - UHE-CANA BRAVA,42.86,50.00,50.00,0.00,0.00
GES - UHE-ESTREITO,7.14,0.00,0.00,0.00,0.00
GES - UHE-ITA,57.14,78.57,78.57,57.14,0.00
GES - UHE-JAGUARA,14.29,78.57,57.14,21.43,0.00
GES - UHE-MACHADINHO,28.57,71.43,71.43,50.00,0.00
GES - UHE-MIRANDA,50.00,57.14,78.57,42.86,0.00
GES - UHE-PASSO FUNDO,21.43,21.43,57.14,50.00,0.00
GES - UHE-PONTE DE PEDRA,0.00,14.29,0.00,0.00,0.00


In [79]:
df2.to_excel('hsMai2024Completo.xlsx')

# Export only the last Month

In [51]:
hs_novo = []
for i in hs_month_export.values():
    hs_novo.append(round(i[-1],2))

In [53]:
coluna = 'Health Score' + ' '+ str(months_number[-1])

In [54]:
df= pd.DataFrame(zip(usinas, hs_novo), columns= ['Usina', coluna])

In [55]:
df

,Usina,Health Score 5
0,GES - PCH-JOSÉ GELÁZIO DA ROCHA,0.00
1,GES - PCH-RONDONÓPOLIS,0.00
2,GES - UHE-CANA BRAVA,0.00
3,GES - UHE-ESTREITO,0.00
4,GES - UHE-ITA,0.00
5,GES - UHE-JAGUARA,0.00
6,GES - UHE-MACHADINHO,0.00
7,GES - UHE-MIRANDA,0.00
8,GES - UHE-PASSO FUNDO,0.00
9,GES - UHE-PONTE DE PEDRA,0.00


In [35]:
df.to_excel('hsMai2024Last.xlsx')